In [1]:
# Install the pycoco library
!pip install pycocotools

  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=272451 sha256=1f895875ccd9b781f0ccb34e563a2808c9532f3f57f69ef2de06da8c81ad09d4
  Stored in directory: /root/.cache/pip/wheels/bc/cf/1b/e95c99c5f9d1648be3f500ca55e7ce55f24818b0f48336adaf
Successfully built pycocotools


In [2]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import os
import random
import cv2
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Define the image and annotation paths
train_imgs_path = '../input/food-recognition-challenge/train-v0.4/train/images'
train_anns_path = '../input/food-recognition-challenge/train-v0.4/train/annotations.json'

test_imgs_path = '../input/food-recognition-challenge/test_images-v0.4/val/images'
test_anns_path = '../input/food-recognition-challenge/test_images-v0.4/val/annotations.json'

# Reference to Image Segmentation

https://divamgupta.com/image-segmentation/2019/06/06/deep-learning-semantic-segmentation-keras.html

In [4]:
# Function to load JSON into a COCO api
def getCOCO(anns_path):
    # Initialize the COCO api for instance annotations
    coco = COCO(anns_path)
    
    # Load the categories in a variable
    catIDs = coco.getCatIds()
    cats = coco.loadCats(catIDs)
    
    # Print number of categories
    nms = [cat['name'] for cat in cats]
    
    # Create a dataframe of the count of each category
    df = pd.DataFrame(columns=['Category', 'ID', 'Count'])
    
    # Add each category and its count row by row
    for i, catID in enumerate(catIDs):
        imgIds = coco.getImgIds(catIds=catID)
        df.loc[i] = [nms[i]] + [catID] + [len(imgIds)]
    
    return coco, df

In [5]:
# Load the annotations of the image dataset
train_coco, train_classes = getCOCO(train_anns_path)
test_coco, test_classes = getCOCO(test_anns_path)

loading annotations into memory...
Done (t=4.29s)
creating index...
index created!
loading annotations into memory...
Done (t=0.16s)
creating index...
index created!


In [6]:
def getMask(image_id, coco, classes, image_size):
    # Create a zero array with the given size and number of classes
    mask = np.zeros((image_size[0], image_size[1], 274))

    annIds = coco.getAnnIds(int(image_id))
    anns = coco.loadAnns(annIds)

    for i, ann in enumerate(anns):
        # Get the binary mask for the annotation
        binary = cv2.resize(coco.annToMask(ann), image_size)

        # Get the channel index for the annotation
        channel = classes[classes.ID == ann['category_id']].index[0] + 1

        # Update the channel of the annotation
        mask[:, :, channel] = binary

        # Update the background channel of the annotation
        if i == 0:
            mask[:, :, 0] = np.logical_or(mask[:, :, 0], np.logical_not(binary))
        else:
            mask[:, :, 0] = np.logical_and(mask[:, :, 0], np.logical_not(binary))

    return mask

In [7]:
def getGenerator(img_folder, coco, classes, image_size, batch_size):
    c = 0
    n = os.listdir(img_folder)
    random.shuffle(n)
    
    while (True):
        img_batch = np.zeros((batch_size, image_size[0], image_size[1], 3)).astype('int')
        mask_batch = np.zeros((batch_size, image_size[0], image_size[1], 274)).astype('float')

        for i in range(c, c + batch_size):
            img = cv2.imread(img_folder + '/' + n[i])
            img =  cv2.resize(img, image_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_batch[i-c] = img

            mask = getMask(n[i][1:-4], coco, classes, image_size)
            mask_batch[i-c] = mask

        c += batch_size
        if(c + batch_size >= len(os.listdir(img_folder))):
            c = 0
            random.shuffle(n)

        yield img_batch, mask_batch

In [8]:
# Get the generators from the paths
IMAGE_SIZE = (32, 32)
BATCH_SIZE = 300

train_gen = getGenerator(train_imgs_path, train_coco, train_classes, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
train_length = len(os.listdir(train_imgs_path))
print('Number of training images: {}'.format(train_length))

test_gen = getGenerator(test_imgs_path, test_coco, test_classes, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
test_length = len(os.listdir(test_imgs_path))
print('Number of test images: {}'.format(test_length))

Number of training images: 24120
Number of test images: 1269


In [9]:
# Install and import the segmentation models library
!pip install segmentation_models
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm

     |████████████████████████████████| 50 kB 1.7 MB/s 
env: SM_FRAMEWORK=tf.keras
Segmentation Models: using `tf.keras` framework.


In [10]:
# Define the model to be trained
model = sm.Unet('resnet34', encoder_freeze=True, classes=274, activation='softmax')
model.compile('Adam', loss=sm.losses.cce_dice_loss, 
              metrics=[sm.metrics.iou_score, sm.metrics.precision, sm.metrics.recall])

85524480/85521592 [==============================] - 1s 0us/step


In [11]:
# # Load the trained model weights
# weights_path = '../input/food-recognition-model/weights.h5'
# model.load_weights(weights_path)

In [12]:
# Train the defined model on the dataset
history = model.fit(train_gen, steps_per_epoch=train_length//BATCH_SIZE, epochs=10)

Epoch 1/10
80/80 [==============================] - 597s 7s/step - loss: 1.0121 - iou_score: 0.0015 - precision: 0.0044 - recall: 0.3584
Epoch 2/10
80/80 [==============================] - 350s 4s/step - loss: 0.9978 - iou_score: 0.0061 - precision: 0.0093 - recall: 0.3692
Epoch 3/10
80/80 [==============================] - 344s 4s/step - loss: 0.9915 - iou_score: 0.0096 - precision: 0.0157 - recall: 0.3763
Epoch 4/10
80/80 [==============================] - 346s 4s/step - loss: 0.9853 - iou_score: 0.0133 - precision: 0.0228 - recall: 0.3834
Epoch 5/10
80/80 [==============================] - 350s 4s/step - loss: 0.9778 - iou_score: 0.0181 - precision: 0.0317 - recall: 0.3935
Epoch 6/10
80/80 [==============================] - 349s 4s/step - loss: 0.9693 - iou_score: 0.0237 - precision: 0.0421 - recall: 0.3954
Epoch 7/10
80/80 [==============================] - 355s 4s/step - loss: 0.9642 - iou_score: 0.0273 - precision: 0.0495 - recall: 0.4014
Epoch 8/10
80/80 [=======================

In [13]:
# Save the model to the output
model.save_weights('weights.h5')

In [14]:
# Evaluate the model on the test generator
scores = model.evaluate(test_gen, steps=test_length//BATCH_SIZE)
print('\nLoss: {}'.format(round(scores[0], 3)))
print('Average IoU: {}'.format(round(scores[1], 3)))
print('Average Precision: {}'.format(round(scores[2], 3)))
print('Average Recall: {}'.format(round(scores[3], 3)))

4/4 [==============================] - 21s 7s/step - loss: 0.9671 - iou_score: 0.0270 - precision: 0.0526 - recall: 0.4123

Loss: 0.967
Average IoU: 0.027
Average Precision: 0.053
Average Recall: 0.412
